In [1]:
import numpy as np
import hapke
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from scipy import optimize
from matplotlib.collections import PatchCollection
from mpl_toolkits import mplot3d
from scipy.interpolate import interp1d
from pyvims import VIMS
from pyvims.misc import MAPS
import random
from pyvims.misc import Map
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from pathlib import Path
from pyvims.wget import wget
plt.rcParams.update({'font.size': 12})

def first_degree(w, a, b):
    return a + b * w

def second_degree(w, a1, b1, c1):
    return a1 + b1 * w + c1 * w ** 2

Load cube (using hapke.retrieve_cube) and optical constants (hapke.opticalconstants)

In [2]:
T = 120
N = 20

opt_c = hapke.opticalconstants(T,sensitivity = N)
n_c = opt_c['n']
k_c = opt_c['k']
wav_c = opt_c['wav']

opt_a = hapke.opticalconstants(T, sensitivity= N,crystallinity=False)
n_a = opt_a['n']
k_a = opt_a['k']
wav_a = opt_a['wav']

int_opt = hapke.inter_optical_constants(wav_c, wav_a, n_c, k_c)

wav = np.array(int_opt['wav'])
n_c = int_opt['n']
k_c = int_opt['k']


In [3]:
file_path = 'C:/Users/USUARIO/Desktop/MSc Thesis/Phase A - Data Analysis/Data/Rhea/R4.txt'
root = 'C:/Users/USUARIO/Desktop/MSc Thesis/Phase A - Data Analysis/Data/Rhea/R4'

cube_R4 = hapke.read_cube(file_path,root)
R4_keys = [key for key in cube_R4]

In [4]:
file_path = 'C:/Users/USUARIO/Desktop/MSc Thesis/Phase A - Data Analysis/Data/Rhea/R1.txt'
root = 'C:/Users/USUARIO/Desktop/MSc Thesis/Phase A - Data Analysis/Data/Rhea/R1'

cube_R1 = hapke.read_cube(file_path,root)
R1_keys = [key for key in cube_R1]

In [5]:
is_pixel_4 = []
pixel_4 = []

lat = 0
lon = 90

radius = cube_R1[R1_keys[0]].target_radius

for key in R4_keys:

    pix = cube_R4[key].get_pixel(lon,lat)

    if pix is not None and hapke.is_micron_dip(pix) and not pix.limb and np.array(pix.spectrum).max() > 0.05 and pix.area < (radius/5) ** 2:
        is_pixel_4.append(True)
        pixel_4.append(pix)
    else:
        is_pixel_4.append(False)
        pixel_4.append(None)

is_pixel_1 = []
pixel_1 = []

for key in R1_keys:

    pix = cube_R1[key].get_pixel(lon,lat)

    if pix is not None and hapke.is_micron_dip(pix) and not pix.limb and np.array(pix.spectrum).max() > 0.05 and pix.area < (radius/5) ** 2:
        is_pixel_1.append(True)
        pixel_1.append(pix)
    else:
        is_pixel_1.append(False)
        pixel_1.append(None)

print(is_pixel_1)
print(is_pixel_4)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, True, True, True, False, True, False, True, False, True, True, True, False, True, True, True, True, False, True, True, False, False, True, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fa

In [ ]:
bg = MAPS['RHEA']
fig, ax = bg.figure(figsize=(20, 10))

patches_1 = []
patches_4 = []
for p in range(len(pixel_4)):
    if is_pixel_4[p]:
        #print(pixel_4[p].corners.lonlat)
        hapke.generate_patches(patches_4,pixel_4[p])

for p in range(len(pixel_1)):
    if is_pixel_1[p]:
        #print(pixel_4[p].corners.lonlat)
        hapke.generate_patches(patches_1,pixel_1[p])

p_1 = PatchCollection(patches_1, alpha=0.4, edgecolor='yellow', linewidth=2)
p_4 = PatchCollection(patches_4, alpha=0.4, edgecolor='red', linewidth=2)

ax.add_collection(p_1)
ax.add_collection(p_4)

In [14]:
plt.show()

KeyboardInterrupt: 

In [18]:
IF_1 = {}
IF_4 = {}
wav1 = {}
wav4 = {}

fig, ax = plt.subplots()
ax.set_xlabel(r'Wavelength [$\mu$m]')
ax.set_ylabel('I/F')
ax.set_title('')
ax.set_xlim(1.90,2.15)
ax.grid()
ax.axvline(x=2.0, color='red', linestyle='--')
ax.axvline(x=2.05, color='red', linestyle='--')


for p in range(len(pixel_1)):
    if is_pixel_1[p]:

        ax.plot(pixel_1[p].wvlns,pixel_1[p].spectrum)

for p in range(len(pixel_4)):
    if is_pixel_4[p]:

        ax.plot(pixel_4[p].wvlns,pixel_4[p].spectrum,label = R4_keys[p], ls = ':')
plt.legend()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


KeyboardInterrupt: 

In [6]:
IF = {}
wav1 = {}
angles = {}

fits = {}
model_IF = {}

D_fit = []
error_D = []
theta_fit = []
error_theta = []
legend = []

cryst_coe = []
cryst_area = []
cryst_area_norm = []
cryst_area_1 = []
cryst_area_norm_1 = []
cryst_2_loc = []

ini_par = [0.7,0.00001]

for p in range(len(is_pixel_4)):
    if is_pixel_4[p]:

        print(str(p) + '/' + str(len(is_pixel_4)))
        IF[R4_keys[p]] = pixel_4[p].spectrum
        wav1[R4_keys[p]] = pixel_4[p].wvlns

        angles[R4_keys[p]] = [np.deg2rad(pixel_4[p].eme),np.deg2rad(pixel_4[p].inc),np.deg2rad(pixel_4[p].phase)]
        print(pixel_4[p].inc)

        fits[R4_keys[p]] = optimize.least_squares(hapke.cost_function_mixed, ini_par, args=(wav, angles[R4_keys[p]], IF[R4_keys[p]], wav1[R4_keys[p]],n_c,k_c,n_a,k_a,0,2.8)
        )

        model_IF[R4_keys[p]] = hapke.hapke_model_mixed(fits[R4_keys[p]].x,wav,angles[R4_keys[p]],n_c,k_c,n_a,k_a)['IF']

        error_D.append(hapke.print_error_correlation(fits[R4_keys[p]])[1])
        error_theta.append(hapke.print_error_correlation(fits[R4_keys[p]])[0])

        legend.append(R4_keys[p])
        D_fit.append(fits[R4_keys[p]].x[1])
        theta_fit.append(fits[R4_keys[p]].x[0])

        cryst_area.append(hapke.cryst_area(IF[R4_keys[p]],wav1[R4_keys[p]])['area'])

        cryst_area_norm.append(hapke.cryst_area(IF[R4_keys[p]],wav1[R4_keys[p]])['area'] /hapke.cryst_area(model_IF[R4_keys[p]],wav)['area'])

        cryst_area_1.append(hapke.cryst_area_one_peak(IF[R4_keys[p]],wav1[R4_keys[p]])['area'])

        cryst_area_norm_1.append(hapke.cryst_area_one_peak(IF[R4_keys[p]],wav1[R4_keys[p]])['area'] /hapke.cryst_area_one_peak(model_IF[R4_keys[p]],wav)['area'])

        cryst_coe.append(hapke.crystallinity_coecient(IF[R4_keys[p]],wav1[R4_keys[p]]))

        cryst_2_loc.append(hapke.cryst_area(IF[R4_keys[p]],wav1[R4_keys[p]])['loc_2'])

for p in range(len(is_pixel_1)):
    if is_pixel_1[p]:

        print(str(p) + '/' + str(len(is_pixel_1)))
        IF[R1_keys[p]] = pixel_1[p].spectrum
        wav1[R1_keys[p]] = pixel_1[p].wvlns

        angles[R1_keys[p]] = [np.deg2rad(pixel_1[p].eme),np.deg2rad(pixel_1[p].inc),np.deg2rad(pixel_1[p].phase)]

        fits[R1_keys[p]] = optimize.least_squares(hapke.cost_function_mixed, ini_par, args=(wav, angles[R1_keys[p]], IF[R1_keys[p]], wav1[R1_keys[p]],n_c,k_c,n_a,k_a,0,2.8)
        )
        print(pixel_1[p].inc)

        model_IF[R1_keys[p]] = hapke.hapke_model_mixed(fits[R1_keys[p]].x,wav,angles[R1_keys[p]],n_c,k_c,n_a,k_a)['IF']

        error_D.append(hapke.print_error_correlation(fits[R1_keys[p]])[1])
        error_theta.append(hapke.print_error_correlation(fits[R1_keys[p]])[0])

        legend.append(R1_keys[p])
        D_fit.append(fits[R1_keys[p]].x[1])
        theta_fit.append(fits[R1_keys[p]].x[0])

        cryst_area.append(hapke.cryst_area(IF[R1_keys[p]],wav1[R1_keys[p]])['area'])

        cryst_area_norm.append(hapke.cryst_area(IF[R1_keys[p]],wav1[R1_keys[p]])['area'] /hapke.cryst_area(model_IF[R1_keys[p]],wav)['area'])

        cryst_area_1.append(hapke.cryst_area_one_peak(IF[R1_keys[p]],wav1[R1_keys[p]])['area'])

        cryst_area_norm_1.append(hapke.cryst_area_one_peak(IF[R1_keys[p]],wav1[R1_keys[p]])['area'] /hapke.cryst_area_one_peak(model_IF[R1_keys[p]],wav)['area'])

        cryst_coe.append(hapke.crystallinity_coecient(IF[R1_keys[p]],wav1[R1_keys[p]]))

        cryst_2_loc.append(hapke.cryst_area(IF[R1_keys[p]],wav1[R1_keys[p]])['loc_2'])



61/129
59.43921537561224
63/129


C:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


60.592166189236224
64/129
58.73487180744921
65/129
79.35764518713925
66/129
76.95319948671333
68/129
71.41989496746606
70/129
68.42090166410081
72/129
79.19803695560898
73/129
78.01581359301619
74/129
76.93183745975536
76/129
69.4175160039405
77/129
67.7221864361156
78/129
83.43061914727444
79/129
81.68999913555523
81/129
71.73296351931468
82/129
69.41675943131418
85/129
79.56666868624843
89/129
79.12087591191776
93/129
83.39957818429428


In [9]:
D = hapke.array_mean(D_fit, error_D)
print(f'D = {D[0]*1000000:.4f} +- {D[1]*1000000:.4f}')

theta = hapke.array_mean(theta_fit, error_theta)
print(f'theta = {theta[0]:.4f} +- {theta[1]:.4f}')

loc_2m = hapke.array_mean(cryst_2_loc)
print(f'Loc 2m = {loc_2m[0]:.4f}')

area_1_peak = hapke.array_mean(cryst_area_1)
print(f'Area 1 peak = {area_1_peak[0]:.4f}')
area_1_peak_norm = hapke.array_mean(cryst_area_norm_1)
print(f'Area 1 peak (norm) = {area_1_peak_norm[0]:.4f}')

area_2_peak = hapke.array_mean(cryst_area)
print(f'Area 2 peak = {area_2_peak[0]:.4f}')
area_2_peak_norm = hapke.array_mean(cryst_area_norm)
print(f'Area 2 peak (norm) = {area_2_peak_norm[0]:.4f}')

D = 108.3613 +-  8.8423
theta = 1.0892 +- 0.0301
Loc 2m = 38.5860
Area 1 peak = 0.0381
Area 1 peak (norm) = 13.5341
Area 2 peak = 0.0606
Area 2 peak (norm) = 12.3573


In [15]:
IF_1 = {}
IF_4 = {}
wav1 = {}
wav4 = {}

fig, ax = plt.subplots()
ax.set_xlabel(r'Wavelength [$\mu$m]')
ax.set_ylabel('I/F')
ax.set_title('')
ax.set_xlim(1.90,2.15)
ax.grid()
ax.axvline(x=2.0, color='red', linestyle='--')
ax.axvline(x=2.05, color='red', linestyle='--')


for p in range(len(pixel_1)):
    if is_pixel_1[p]:

        ax.plot(pixel_1[p].wvlns,pixel_1[p].spectrum)

for p in range(len(pixel_4)):
    if is_pixel_4[p]:

        ax.plot(pixel_4[p].wvlns,pixel_4[p].spectrum,label = R4_keys[p])
w = np.linspace(1.9,2.15,20)
for i in range(len(cryst_2_loc)):
    ax.plot(w,second_degree(w,cryst_2_loc[i]['fit2'][0],cryst_2_loc[i]['fit2'][1],cryst_2_loc[i]['fit2'][2]), ls = ':')

plt.legend()
plt.show()

KeyboardInterrupt: 

In [21]:
fig, (ax1,ax2) = plt.subplots(2,1)

ax1.errorbar(legend, np.array(D_fit)*1000000,yerr=np.array(error_D)*1000000, fmt='-o')
ax1.set_ylabel(r'D')

ax2.errorbar(legend, np.array(theta_fit),yerr=np.array(error_theta), fmt='-o')
ax2.set_ylabel(r'$\bar{\theta}$')
ax2.set_xlabel(r'Experiment')
ax1.grid()
ax2.grid()
plt.legend()

plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


KeyboardInterrupt: 

In [ ]:
print(legend)

In [10]:
fig, (ax1,ax2,ax3) = plt.subplots(3,1)

ax1.errorbar(legend, np.array(cryst_2_loc), fmt='-o')
ax1.set_ylabel(r'Loc 2 micron peak')

ax2.errorbar(legend, np.array(cryst_area_1), fmt='-o')
ax2.set_ylabel(r'Cryst Area')

ax3.errorbar(legend, np.array(cryst_area_norm_1), fmt='-o')
ax3.set_ylabel(r'Cryst Area Normalized')

ax1.grid()
ax2.grid()
ax3.grid()

plt.show()

KeyboardInterrupt: 

In [ ]:
cube = cube_R1['1511675722_1']

In [ ]:
bg = MAPS[cube.target_name]
fig, ax = bg.figure(figsize=(20, 10))

ax.add_collection(cube.pixels(2.03, vmin=.05, vmax=.18))
plt.show()

In [ ]:
pixel = cube.get_pixel(25,8)

ax = hapke.plot_pixel_equi(2.03,cube,[(pixel.s,pixel.l)],background=True)

plt.show()

In [ ]:
pixel1 = cube@(13, 50)

e, i, phase = [np.deg2rad(pixel1.eme),np.deg2rad(pixel1.inc),np.deg2rad(pixel1.phase)]


angles = [e,i,phase]
IF1 = pixel1.spectrum
wav1 = pixel1.wvlns
ini_par = [0.1,0.00001]

body = 0

optimized_parameters = optimize.least_squares(
    hapke.cost_function_mixed, ini_par, args=(wav, angles, IF1, wav1,n_c,k_c,n_a,k_a,0,2.8)
)

optimized_values = optimized_parameters.x
hapke.print_error_correlation(optimized_parameters)

print('')

ini_mass = 0.1

cryst_fit = optimize.least_squares(
        hapke.cost_function_mixed_mass_fraction, ini_mass, args=(wav, angles, IF1, wav1,n_c,k_c,n_a,k_a,optimized_values,1.45,1.8,1.8), bounds=([0],[1]),
)

hapke.print_error_correlation(cryst_fit)

print('1.2-1.65 RATIO: ' + str(hapke.crystallinity_coecient(IF1,wav1)))

print('')
print('Longitude: ' + str(pixel1.lon))
print('Latitude: ' + str(pixel1.lat))
print('Phase: ' + str(pixel1.phase))

In [ ]:
print(hapke.cryst_area(IF1,wav1)['area'])
print(hapke.cryst_area(hapke.hapke_model_mixed(optimized_values, wav, angles, n_c, k_c, n_a, k_a)['IF'],wav)['area'])
print(hapke.cryst_area(IF1,wav1)['area']/hapke.cryst_area(hapke.hapke_model_mixed(optimized_values, wav, angles, n_c, k_c, n_a, k_a)['IF'],wav)['area'])

In [ ]:
fig, ax = plt.subplots()
ax.plot(wav1, IF1)
ax.plot(wav, hapke.hapke_model_mixed(optimized_values, wav, angles, n_c, k_c, n_a, k_a)['IF'], ls = ':')
ax.plot(wav, hapke.hapke_model_mixed_mass_fraction(cryst_fit.x, wav, angles, n_c, k_c, n_a, k_a,optimized_values)['IF'], ls = '-.')
ax.set_xlabel(r'Wavelength [$\mu$m]')
ax.set_ylabel('I/F')
ax.set_title('')
ax.set_xlim(1.1,3.3)
ax.grid()
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#ax.legend()
plt.show()

In [ ]:
pixel1 = cube@(30,15)

e, i, phase = [np.deg2rad(pixel1.eme),np.deg2rad(pixel1.inc),np.deg2rad(pixel1.phase)]


angles = [e,i,phase]
IF1 = pixel1.spectrum
wav1 = pixel1.wvlns
ini_par = [0.1,np.deg2rad(20),0.1,0.00005,0.5]

body = cube.target_name

optimized_parameters = optimize.least_squares(
    hapke.cost_function_mixed_no_weight, ini_par, args=(wav, angles, IF1, wav1,n_c,k_c,n_a,k_a,1.5,1.85,body), bounds=([0.001,0,0,0,0], [1,np.deg2rad(44.9),0.74,0.001,1], ), x_scale='jac',
)

optimized_values = optimized_parameters.x
hapke.print_error_correlation(optimized_parameters)

print('')
print('Longitude: ' + str(pixel1.lon))
print('Latitude: ' + str(pixel1.lat))
print('Phase: ' + str(pixel1.phase))

This cell below created 20 randomly generated initial parameters and store the results of the fits on a dictionary fit_dict. The variable count adds one every time a set of generated initial parameters leads to the same solution than the previous one.

In [ ]:
fit_dict = {}
count = 0
n_fits = 20

for i in range(n_fits):
    param = [random.random()*0.74,random.random()*0.001,random.random()]
    optimized_parameters = optimize.least_squares(
        hapke.cost_function_mixed_no_weight, param, args=(wav, angles, IF1, wav1,n_c,k_c,n_a,k_a,0,2.8,body), bounds=([0.00001,0.000001,0], [0.74,0.001,1], ),
    )

    fit_dict[i] = {'x': optimized_parameters.x, 'cost': optimized_parameters.cost}

    if i == 0:

        opt_param =  optimized_parameters.x
        opt_cost = optimized_parameters.cost
        count += 1
        print(optimized_parameters.x)
        print('COST = ' + str(optimized_parameters.cost))

    elif i >= i:

        rel_er1 = abs(opt_param[0] - optimized_parameters.x[0]) / opt_param[0]
        rel_er2 = abs(opt_param[1] - optimized_parameters.x[1]) / opt_param[1]
        rel_er3 =abs(opt_param[2] - optimized_parameters.x[2]) / opt_param[2]

        if rel_er1 > 0.0001 or rel_er2 > 0.0001 or rel_er3 > 0.0001:
            print('PARAMETERS CHANGED')
            best_fit =  optimized_parameters
            print(optimized_parameters.x)
            print('COST = ' + str(optimized_parameters.cost))
        else:
            count += 1
print(count)

In [ ]:
fit_dict = {}
count = 0
n_fits = 20

for i in range(n_fits):
    param = [random.random(),random.random()*np.deg2rad(44.5),random.random()*0.74,random.random()*0.001,random.random()]
    optimized_parameters = optimize.least_squares(
        hapke.cost_function_mixed_no_weight, param, args=(wav, angles, IF1, wav1,n_c,k_c,n_a,k_a,0,2.8,body), bounds=([0.001,0,0,0,0], [1,np.deg2rad(44.9),0.74,0.001,1], ),
    )

    fit_dict[i] = {'x': optimized_parameters.x, 'cost': optimized_parameters.cost}

    if i == 0:

        opt_param =  optimized_parameters.x
        opt_cost = optimized_parameters.cost
        count += 1
        print(optimized_parameters.x)
        print('COST = ' + str(optimized_parameters.cost))

    elif i >= i:

        rel_er1 = abs(opt_param[0] - optimized_parameters.x[0]) / opt_param[0]
        rel_er2 = abs(opt_param[1] - optimized_parameters.x[1]) / opt_param[1]
        rel_er3 =abs(opt_param[2] - optimized_parameters.x[2]) / opt_param[2]
        rel_er4 = abs(opt_param[3] - optimized_parameters.x[3]) / opt_param[3]
        rel_er5 =abs(opt_param[4] - optimized_parameters.x[4]) / opt_param[4]

        if rel_er1 > 0.0001 or rel_er2 > 0.0001 or rel_er3 > 0.0001 or rel_er4 > 0.0001 or rel_er5 > 0.0001:
            print('PARAMETERS CHANGED')
            best_fit =  optimized_parameters
            print(optimized_parameters.x)
            print('COST = ' + str(optimized_parameters.cost))
        else:
            count += 1
print(count)

In [ ]:
print(fit_dict)

In [ ]:
optimized_values = optimized_parameters.x

old_hapke = hapke.hapke_model_mixed(ini_par,wav, angles, n_c,k_c,n_a,k_a,body)['IF']

new_hapke = hapke.hapke_model_mixed(optimized_values,wav, angles, n_c,k_c,n_a,k_a,body)['IF']

fig, ax = plt.subplots(figsize = (8,3))
ax.plot(wav, old_hapke, label = 'Hapke Initial')
ax.plot(wav, new_hapke, label = 'Hapke Optimized')
ax.plot(wav1, IF1, label = 'Pixel')
ax.set_xlabel('Wavelength [$\mu$m]')
ax.set_ylabel('I/F')
ax.set_xlim(1.1,3.2)
ax.set_title('')
ax.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

plt.imshow(cube@2.03, extent=cube.extent, cmap='gray', vmin=0, vmax=.18)

plt.colorbar(extend='max', label='I/F')

plt.scatter(13, 50, s=150)

plt.xlabel(cube.slabel)
plt.ylabel(cube.llabel)

plt.xticks(cube.sticks)
plt.yticks(cube.lticks)
plt.show()

These coming two cells plot the surface response: the residuals compared to the spectrum taken by VIMS. Note that wa1 and IF1 need to be defined beforehand and that the function hapke_mixed needs to be changes to be 2D in parameters

In [ ]:
# SURFACE RESPONSE OF ENCELADUS SPECTRA

real_par = [0.51,0.00007]

m_range = np.linspace(0.001,1,30)
phi_range = np.linspace(0.001,0.74,30)

residuals = np.zeros((len(phi_range),len(m_range)))


interp_func_2 = interp1d(wav1, IF1, bounds_error=False)

interpolated_lab = interp_func_2(wav)

for i in range(len(phi_range)):
    for j in range(len(m_range)):
        param = [phi_range[i], m_range[j]]
        dif = (interpolated_lab - hapke.hapke_model_mixed(param,wav,angles,n_c,k_c,n_a,k_a,body)['IF'])
        dif = dif[~np.isnan(dif)]
        residuals[i,j] = np.linalg.norm(dif)

In [ ]:
# 3D PLOT OF THE SURFACE RESPONSE

phi_range, m_range = np.meshgrid(phi_range, m_range)
# Define the data

# Create a 3D plot
fig = plt.figure(figsize = (4,4))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(phi_range, m_range, residuals.T, cmap='viridis')
ax.scatter(0.16933,0.502, c = 'green', label = 'Real solution' )
ax.legend()
ax.set_xlabel(r'$\phi$ [-]')
ax.set_ylabel(r'$m$ [-]')
ax.set_zlabel('Residuals')
# Show the plot
plt.show()